<a href="https://colab.research.google.com/github/amaliarachma25/NOAA-Coral-Bleaching-Monitoring/blob/main/NOAA_CRW_Server_Otomasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌊 Automated Pipeline: NOAA Coral Reef Watch Data Processing

Project: Monitoring Kesehatan Karang (SST, DHW, SSTA)

Notebook ini berfungsi untuk mengunduh data satelit NOAA secara otomatis, memotong area spesifik (Selat Lombok), dan mengekstrak data site berdasarkan batas kawasan konservasi (Shapefile).


📋 Alur Kerja (Workflow):

1.   Setup: Menyiapkan folder otomatis di Google Drive.
2.   Download: Mengambil data harian global dari Server NOAA.
3.   Clip Regional: Memotong data global menjadi area kecil (Lombok).
4.   Masking Site: Memotong spesifik sesuai bentuk kawasan (Gili Matra, Gita Nada, Nusa Penida) dan convert ke format .xyz.



# > ⚠️ WAJIB DILAKUKAN SEBELUM MULAI:

Pastikan sudah mengupload folder Shapefile (.shp, .shx, .dbf, dll) ke dalam Google Drive.

Lokasi: /MAGANG/CORAL/SHP_SITE/

File yang dibutuhkan:

*   gili_matra_buffer_5km.shp
*   gita_nada_buffer_5km.shp
* nusa_penida_buffer_5km.shp

# 📂 Struktur Folder Proyek (Mind Map) (contoh)

Pastikan struktur folder di Google Drive (`My Drive/MAGANG/CORAL/...`) sesuai dengan peta di bawah ini agar script berjalan lancar.

```text
My Drive/
└── MAGANG/
    └── CORAL/  (Folder Utama Proyek)
        │
        ├── [ALUR 1: PENGOLAHAN DATA HARIAN]
        │   ├── 01_Global/                     # [Input]  Tempat hasil download NOAA Daily (.nc)
        │   ├── 02_Clip_Lombok/                # [Proses] File .nc harian dipotong khusus area Lombok
        │   ├── SHP_SITE/                      # [Manual] WAJIB Upload Shapefile (.shp) di sini
        │   └── 03_masking_site/               # [Output] Hasil ekstraksi koordinat (.xyz) per site
        │
        ├── [ALUR 2: PENGOLAHAN CLIMATOLOGY]
        │   └── 01_climatology/
        │       ├── SHP_SITE/                  # [Manual] Copy Shapefile yang sama ke folder ini
        │       ├── 00_raw_climatology/        # [Input]  Tempat hasil download Climatology (.nc)
        │       ├── 03_masking_site_clim/      # [Output] Hasil ekstraksi Climatology per site
        │       └── average_climatology.txt    # [Result] File txt berisi nilai MMM & Mean Bulanan
        │
        └── [ALUR 3: ANALISIS AKHIR]
            └── NOAA_Final_Reports # [Final]  Laporan Akhir (BAA, DHW, Percentile)
                                               # (Menggabungkan data dari 03_masking_site & average_climatology.txt)


#Step 1 Persiapan Environment

Tahap ini akan menginstal alat bantu (library) yang diperlukan untuk mengolah data satelit (xarray, netCDF4) dan menghubungkan Google Colab dengan Google Drive.

Apa yang dilakukan script ini?

Menginstall library Python.

Membuat struktur folder otomatis di Drive :

*   01_Global (Untuk data mentah)
*   02_Clip_Lombok (Untuk hasil potongan kasar)

In [1]:
from google.colab import drive

# 1. Mount Drive
# Ini akan memunculkan popup meminta izin akses ke Akun Google kamu.
# Klik "Connect to Google Drive" lalu pilih akunmu dan klik "Allow/Izinkan".
drive.mount('/content/drive')

# 2. Cek apakah berhasil
import os
if os.path.exists('/content/drive'):
    print("✅ Google Drive berhasil terhubung!")
else:
    print("❌ Gagal terhubung.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive berhasil terhubung!


In [2]:
# 1. Install Library yang dibutuhkan (xarray & netCDF4 biasanya perlu install dulu di Colab)
!pip install xarray netCDF4 requests

# 2. Buat Folder Penyimpanan Sementara di Colab
import os

# Define the base directory on Google Drive
base_drive_dir = "/content/drive/MyDrive/MAGANG/CORAL" #diganti sesuai drive sendiri

# Ensure the base directory exists
if not os.path.exists(base_drive_dir):
    try:
        os.makedirs(base_drive_dir, exist_ok=True)
        print(f"Created base directory: {base_drive_dir}")
    except OSError as e:
        print(f"❌ Error creating base directory {base_drive_dir}: {e}")

# Folder untuk menyimpan file mentah (Raw)
raw_dir = os.path.join(base_drive_dir, "01_Global") #Buat folder di drive untuk menyimpan hasil global
if not os.path.exists(raw_dir):
    os.makedirs(raw_dir, exist_ok=True)

# Folder untuk menyimpan hasil potongan (Clip)
clip_dir = os.path.join(base_drive_dir, "02_Clip_Lombok") #Buat folder di drive untuk menyimpan hasil clip
if not os.path.exists(clip_dir):
    os.makedirs(clip_dir, exist_ok=True)

print("✅ Folder siap digunakan!")
print(f"📂 Lokasi Download: {raw_dir}")
print(f"📂 Lokasi Output: {clip_dir}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.8 MB/s eta 0:00:00
✅ Folder siap digunakan!
📂 Lokasi Download: /content/drive/MyDrive/MAGANG/CORAL/01_Global
📂 Lokasi Output: /content/drive/MyDrive/MAGANG/CORAL/02_Clip_Lombok


#Step 2 Download (Direct Server)

Script ini bertugas mengambil data mentah langsung dari server NOAA Coral Reef Watch. Data yang diambil mencakup:

SST: Suhu Permukaan Laut.

SSTA: Anomali Suhu (Penyimpangan dari rata-rata).

HS: HotSpot (Stress panas harian).

DHW: Degree Heating Weeks (Akumulasi stress panas/risiko bleaching).

Catatan:

*   Atur tanggal pada variabel start_date dan end_date sesuai kebutuhan.
*   Data akan tersimpan otomatis di folder 01_Global.

In [4]:
import requests
import os
from datetime import datetime, timedelta
import time

# --- KONFIGURASI ---
# Kita gunakan folder yang sudah dibuat di Langkah 0
save_dir = "/content/drive/MyDrive/MAGANG/CORAL/01_Global"

# RENTANG TANGGAL
start_date = datetime(2025, 12, 31)
end_date   = datetime(2025, 12, 31)

# URL & CONFIG
base_url = "https://www.star.nesdis.noaa.gov/pub/socd/mecb/crw/data/5km/v3.1_op/nc/v1.0/daily"

var_config = {
    "sst":  ["sst",  "coraltemp",  "NOAA_SST"],
    "ssta": ["ssta", "ct5km_ssta", "NOAA_SSTA"],
    "hs":   ["hs",   "ct5km_hs",   "NOAA_HS"],
    "dhw":  ["dhw",  "ct5km_dhw",  "NOAA_DHW"]
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

def download_step():
    print(f"--- MULAI DOWNLOAD DATA ---")
    print(f"Target: {start_date.strftime('%Y-%m-%d')} s/d {end_date.strftime('%Y-%m-%d')}")

    current_date = start_date
    while current_date <= end_date:
        date_str_url = current_date.strftime("%Y%m%d")
        year = current_date.strftime("%Y")
        date_disp = current_date.strftime('%Y-%m-%d')

        print(f"[{date_disp}] Memproses...", end=" ")

        for var_key, config in var_config.items():
            folder_server = config[0]
            prefix_server = config[1]
            prefix_local  = config[2]

            local_filename = f"{prefix_local}_{date_str_url}.nc"
            local_path = os.path.join(save_dir, local_filename)

            # Cek jika sudah ada
            if os.path.exists(local_path) and os.path.getsize(local_path) > 1000:
                continue

            # Download
            filename_server = f"{prefix_server}_v3.1_{date_str_url}.nc"
            url = f"{base_url}/{folder_server}/{year}/{filename_server}"

            try:
                response = requests.get(url, headers=headers, stream=True, timeout=60)
                if response.status_code == 200:
                    with open(local_path, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=1024*1024):
                            f.write(chunk)
                else:
                    print(f"[x] Gagal {var_key.upper()}: {response.status_code}")
            except Exception as e:
                print(f"[!] Error {var_key.upper()}: {e}")

            time.sleep(0.2) # Jeda sopan

        print("Selesai.")
        current_date += timedelta(days=1)

    print("\n✅ Download Selesai. Cek folder di menu file sebelah kiri.")

# Jalankan Fungsi
download_step()

--- MULAI DOWNLOAD DATA ---
Target: 2025-12-31 s/d 2025-12-31
[2025-12-31] Memproses... Selesai.

✅ Download Selesai. Cek folder di menu file sebelah kiri.


# Step 3 Clipping (Selat Lombok)
Mengolah data global (seluruh dunia) sangat berat dan lambat. Tahap ini akan memotong data global tersebut hanya pada kotak koordinat Selat Lombok dan sekitarnya. Memperkecil ukuran file secara drastis, mempercepat proses masking di tahap selanjutnya. Hasil potongan akan disimpan di folder 02_Clip_lombok.

Setup & Install Library

In [5]:
!pip install xarray netCDF4

import xarray as xr
import os
import glob
import warnings
import shutil
import zipfile
from google.colab import files
from google.colab import drive

# 1. Hubungkan Google Drive (Jika belum)
drive.mount('/content/drive')

# --- 2. KONFIGURASI FOLDER INPUT (LANGSUNG DARI DRIVE) ---
# Sesuaikan path ini dengan lokasi folder kamu di Google Drive.
# Biasanya formatnya: "/content/drive/MyDrive/NAMA_FOLDER_KAMU/..."
# Contoh: Jika di laptop D:\magang\CORAL\output\01_hasil_noaa_global
# Maka di Drive biasanya:

input_dir = "/content/drive/MyDrive/MAGANG/CORAL/01_Global"
# ^^^ GANTI PATH DI ATAS JIKA NAMA FOLDER DI DRIVE KAMU BERBEDA ^^^

# Folder Output (Simpan hasil clip ke Drive juga biar aman)
output_dir = "/content/drive/MyDrive/MAGANG/CORAL/02_Clip_lombok"

# Buat folder output jika belum ada
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- 3. CEK APAKAH FILE TERBACA? ---
# Kita cek apakah script bisa melihat file .nc di folder itu
nc_files = glob.glob(os.path.join(input_dir, "*.nc"))

print(f"📂 Folder Input: {input_dir}")
print(f"📂 Folder Output: {output_dir}")
print(f"🔍 Ditemukan {len(nc_files)} file .nc")

if len(nc_files) == 0:
    print("❌ PERINGATAN: Tidak ada file .nc ditemukan! Cek lagi path 'input_dir' di atas.")
else:
    print("✅ File siap diproses! Lanjut ke script clipping di bawah.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Folder Input: /content/drive/MyDrive/MAGANG/CORAL/01_Global
📂 Folder Output: /content/drive/MyDrive/MAGANG/CORAL/02_Clip_lombok
🔍 Ditemukan 5 file .nc
✅ File siap diproses! Lanjut ke script clipping di bawah.


Script Clipping (Lombok Strait)

In [6]:
import xarray as xr
import warnings

# Abaikan warning
warnings.filterwarnings("ignore")

# --- KOORDINAT SELAT LOMBOK ---
# [115.2, 116.2, -9, -8]
lat_min, lat_max = -9, -8
lon_min, lon_max = 115.2, 116.2

def process_direct_from_drive():
    print("--- BATCH CLIPPING LOMBOK (DIRECT DRIVE) ---")

    # Ambil list file yang sudah dicek tadi
    all_files = glob.glob(os.path.join(input_dir, "*.nc"))
    total_files = len(all_files)

    if total_files == 0:
        return

    success_count = 0
    fail_count = 0

    for i, file_path in enumerate(all_files, 1):
        filename = os.path.basename(file_path)

        # Skip jika file hasil clip (menghindari loop)
        if filename.startswith("Clip_"):
            continue

        print(f"[{i}/{total_files}] Memproses: {filename} ... ", end='', flush=True)

        try:
            ds = xr.open_dataset(file_path)

            # --- DETEKSI NAMA VARIABEL ---
            if 'lat' in ds.coords:
                lat_name, lon_name = 'lat', 'lon'
            elif 'latitude' in ds.coords:
                lat_name, lon_name = 'latitude', 'longitude'
            else:
                print(" -> ERROR (Koordinat?)")
                continue

            # --- SORTING & SLICING ---
            ds = ds.sortby([lat_name, lon_name])

            # Lakukan Clipping Area Lombok
            ds_clipped = ds.sel({
                lat_name: slice(lat_min, lat_max),
                lon_name: slice(lon_min, lon_max)
            })

            # Cek hasil
            if ds_clipped.dims[lat_name] == 0 or ds_clipped.dims[lon_name] == 0:
                print(" -> GAGAL (Kosong)")
                fail_count += 1
            else:
                # Simpan LANGSUNG ke Google Drive
                output_filename = f"Clip_Lombok_{filename}"
                output_path = os.path.join(output_dir, output_filename)

                ds_clipped.to_netcdf(output_path)
                print(" -> SUKSES!")
                success_count += 1

            ds_clipped.close()
            ds.close()

        except Exception as e:
            print(f" -> ERROR: {e}")
            fail_count += 1

    print("\n" + "="*30)
    print(f"SELESAI. Cek folder '{output_dir}' di Google Drive kamu.")

# Jalankan Fungsi
process_direct_from_drive()

--- BATCH CLIPPING LOMBOK (DIRECT DRIVE) ---
[1/5] Memproses: NOAA_SST_20251202.nc ...  -> SUKSES!
[2/5] Memproses: NOAA_SST_20251231.nc ...  -> SUKSES!
[3/5] Memproses: NOAA_SSTA_20251231.nc ...  -> SUKSES!
[4/5] Memproses: NOAA_HS_20251231.nc ...  -> SUKSES!
[5/5] Memproses: NOAA_DHW_20251231.nc ...  -> SUKSES!

SELESAI. Cek folder '/content/drive/MyDrive/MAGANG/CORAL/02_Clip_lombok' di Google Drive kamu.


Download Hasil(opsional)

In [7]:
# Zip folder output
shutil.make_archive("/content/drive/MyDrive/MAGANG/CORAL/02_Clip_lombok", 'zip', output_dir)

# Download ke laptop
files.download("/content/drive/MyDrive/MAGANG/CORAL/02_Clip_lombok.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Step 4 Clipping (Masked Site)
memotong data menggunakan Shapefile (Peta Batas Kawasan) agar data yang diambil benar-benar hanya yang berada di dalam kawasan konservasi:


1.   Gili Matra (Gili Meno, Air, Trawangan)
2.   Gita Nada (Gili Tangkong, Nanggu, Sudak)
3.  Nusa Penida


Output Akhir:
File berformat .xyz (Text file berisi Longitude, Latitude, Value) yang siap di-plot atau dianalisis lebih lanjut. File ini juga akan otomatis di-ZIP agar mudah di-download.

Setup & Install Library

In [12]:
# 1. Install Library GeoSpasial
!pip install geopandas rioxarray netCDF4

# 2. Mount Google Drive
from google.colab import drive
import os
import glob
import xarray as xr
import rioxarray
import geopandas as gpd
import pandas as pd
import warnings

# Abaikan warning
warnings.filterwarnings("ignore")

drive.mount('/content/drive')

# --- KONFIGURASI PATH UTAMA ---
# Path dasar project kamu di Drive
base_project_dir = "/content/drive/MyDrive/MAGANG/CORAL"

# Folder Input (.nc yang sudah di-clip Lombok)
input_nc_dir = os.path.join(base_project_dir, "02_Clip_lombok")

# Folder Output (.xyz)
output_dir = os.path.join(base_project_dir, "03_Masking_Site")

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"📂 Input: {input_nc_dir}")
print(f"📂 Output: {output_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Input: /content/drive/MyDrive/MAGANG/CORAL/02_Clip_lombok
📂 Output: /content/drive/MyDrive/MAGANG/CORAL/03_Masking_Site


Konfigurasi Shapefile (PENTING)

In [13]:
# --- KONFIGURASI LOKASI SHAPEFILE ---

# Sesuaikan path ini dengan lokasi folder "basemap" kamu di Google Drive
# Asumsi: Kamu mengupload folder 'bahan' ke dalam folder 'CORAL' di Drive (di sini folder site namanya SHP_SITE)
shp_base_dir = os.path.join(base_project_dir, "SHP_SITE")

# Cek apakah folder shp ada?
if not os.path.exists(shp_base_dir):
    print(f"⚠️ PERINGATAN: Folder Shapefile tidak ditemukan di:\n{shp_base_dir}")
    print("👉 Tolong edit variabel 'shp_base_dir' di atas sesuai lokasi folder shp di Drive kamu.")
else:
    print(f"✅ Folder Shapefile ditemukan!")

# Dictionary Shapefile
# Kita gabungkan folder dasar dengan nama filenya
regions = {
    "gm": os.path.join(shp_base_dir, "gili_matra_buffer_5km.shp"),
    "gn": os.path.join(shp_base_dir, "gita_nada_buffer_5km.shp"),
    "np": os.path.join(shp_base_dir, "nusa_penida_buffer_5km.shp")
}

✅ Folder Shapefile ditemukan!


Jalankan Proses Masking

In [14]:
def masking_process_colab():
    print("--- PROSES MASKING SHAPEFILE KE XYZ (DRIVE) ---")

    # Ambil semua file .nc
    nc_files = glob.glob(os.path.join(input_nc_dir, "*.nc"))

    if not nc_files:
        print("[ERROR] Tidak ada file .nc di folder input! Cek path folder 02_Clip_lombok.")
        return

    # Loop setiap file Raster (.nc)
    for i, nc_path in enumerate(nc_files, 1):
        filename_full = os.path.basename(nc_path)
        filename_no_ext = os.path.splitext(filename_full)[0] # Hilangkan .nc

        # Filter file yang mau diolah
        # Kita hanya ingin mengolah file yang namanya "Clip_Lombok_..." atau file NOAA asli
        # Skip jika file itu file sisa/temp
        if filename_full.startswith("Layer_"):
            continue

        print(f"[{i}/{len(nc_files)}] Memproses: {filename_full}")

        try:
            # 1. Buka Raster
            ds = xr.open_dataset(nc_path)

            # Deteksi nama koordinat
            if 'lat' in ds.coords:
                ds = ds.rio.set_spatial_dims("lon", "lat")
                x_name, y_name = 'lon', 'lat'
            elif 'latitude' in ds.coords:
                ds = ds.rio.set_spatial_dims("longitude", "latitude")
                x_name, y_name = 'longitude', 'latitude'
            else:
                print("   -> Skip (Koordinat tidak dikenali)")
                continue

            # Set CRS & Sorting (Agar aman saat clipping)
            ds = ds.rio.write_crs("EPSG:4326", inplace=True)
            ds = ds.sortby([y_name, x_name])

            # Loop setiap Wilayah (Shapefile)
            for code, shp_path in regions.items():
                # Cek dulu apakah file shp beneran ada
                if not os.path.exists(shp_path):
                    print(f"   -> [SKIP] File SHP tidak ditemukan: {shp_path}")
                    continue

                try:
                    # 2. Buka Shapefile
                    gdf = gpd.read_file(shp_path)

                    # Samakan CRS
                    if gdf.crs != ds.rio.crs:
                        gdf = gdf.to_crs(ds.rio.crs)

                    # 3. Lakukan Clipping (Masking)
                    try:
                        clipped = ds.rio.clip(gdf.geometry, gdf.crs, drop=True)
                    except Exception as e_clip:
                        # Kadang error jika tidak ada overlap sama sekali
                        print(f"   -> {code}: Tidak overlap/diluar area raster.")
                        continue

                    # 4. Konversi ke Format XYZ
                    df = clipped.to_dataframe().reset_index()

                    # Cari variabel data
                    ignore_cols = [x_name, y_name, 'time', 'spatial_ref', 'crs', 'band']
                    data_vars = [col for col in df.columns if col not in ignore_cols]

                    if not data_vars:
                        print(f"   -> Warning: Tidak ada data variabel di {code}")
                        continue

                    target_var = data_vars[0]

                    # Filter X, Y, Z dan drop NaN
                    xyz_df = df[[x_name, y_name, target_var]].dropna()

                    if xyz_df.empty:
                        print(f"   -> {code}: Hasil Kosong (Semua NaN)")
                        continue

                    # 5. Simpan ke .XYZ di Drive
                    # Nama file output sesuai request
                    output_filename = f"{code}_{filename_no_ext}.xyz"
                    output_path = os.path.join(output_dir, output_filename)

                    xyz_df.to_csv(output_path, sep=' ', header=False, index=False)

                    print(f"   -> OK: {output_filename}")

                except Exception as e_shp:
                    print(f"   -> Error wilayah {code}: {e_shp}")

            ds.close()

        except Exception as e_file:
            print(f"   -> Gagal membuka file: {e_file}")

    print("\n--- SELESAI SEMUA ---")
    print(f"Cek folder output di: {output_dir}")

# Jalankan Fungsi
masking_process_colab()

--- PROSES MASKING SHAPEFILE KE XYZ (DRIVE) ---
[1/5] Memproses: Clip_Lombok_NOAA_SST_20251202.nc
   -> OK: gm_Clip_Lombok_NOAA_SST_20251202.xyz
   -> OK: gn_Clip_Lombok_NOAA_SST_20251202.xyz
   -> OK: np_Clip_Lombok_NOAA_SST_20251202.xyz
[2/5] Memproses: Clip_Lombok_NOAA_SST_20251231.nc
   -> OK: gm_Clip_Lombok_NOAA_SST_20251231.xyz
   -> OK: gn_Clip_Lombok_NOAA_SST_20251231.xyz
   -> OK: np_Clip_Lombok_NOAA_SST_20251231.xyz
[3/5] Memproses: Clip_Lombok_NOAA_SSTA_20251231.nc
   -> OK: gm_Clip_Lombok_NOAA_SSTA_20251231.xyz
   -> OK: gn_Clip_Lombok_NOAA_SSTA_20251231.xyz
   -> OK: np_Clip_Lombok_NOAA_SSTA_20251231.xyz
[4/5] Memproses: Clip_Lombok_NOAA_HS_20251231.nc
   -> OK: gm_Clip_Lombok_NOAA_HS_20251231.xyz
   -> OK: gn_Clip_Lombok_NOAA_HS_20251231.xyz
   -> OK: np_Clip_Lombok_NOAA_HS_20251231.xyz
[5/5] Memproses: Clip_Lombok_NOAA_DHW_20251231.nc
   -> OK: gm_Clip_Lombok_NOAA_DHW_20251231.xyz
   -> OK: gn_Clip_Lombok_NOAA_DHW_20251231.xyz
   -> OK: np_Clip_Lombok_NOAA_DHW_20251231.x

Download Hasil

In [ ]:
import shutil
from google.colab import files

# Zip folder output
shutil.make_archive("/content/drive/MyDrive/MAGANG/CORAL/03_Masking_Site", 'zip', output_dir)

# Download
files.download("/content/drive/MyDrive/MAGANG/CORAL/03_Masking_Site.zip")

#NOAA Climatology

### Tahap 1: Download & Clipping Data Climatology
**Tujuan:** Mengunduh data Climatology NOAA dan memotongnya sesuai area Shapefile.

⚠️ **WAJIB CEK SEBELUM RUN:**
Pastikan Anda sudah mengupload file Shapefile lengkap (`.shp`, `.shx`, `.dbf`, `.prj`) ke dalam folder:
`.../MAGANG/CORAL/01_climatology/SHP_SITE/`

Jika folder kosong, script akan error atau men-skip proses clipping.

In [4]:
# Jalankan ini dulu untuk menginstall library yang dibutuhkan di Colab
!pip install xarray rioxarray geopandas netCDF4

In [5]:
import os
import requests
import glob
import warnings
import xarray as xr
import rioxarray
import geopandas as gpd
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Abaikan warning
warnings.filterwarnings("ignore")

# ==========================================
# 1. KONFIGURASI PATH (GOOGLE DRIVE)
# ==========================================

# Path disesuaikan untuk Google Colab
# Pastikan folder 'MAGANG' ada di 'My Drive' kamu
base_dir = "/content/drive/MyDrive/MAGANG/CORAL/01_climatology"

# Folder Output & Input
input_global_dir = os.path.join(base_dir, "00_Raw_Climatology")
output_xyz_dir = os.path.join(base_dir, "03_Masking_Site_Clim")
shp_base_dir = os.path.join(base_dir, "SHP_SITE")

# Buat folder jika belum ada
os.makedirs(input_global_dir, exist_ok=True)
os.makedirs(output_xyz_dir, exist_ok=True)

# Link NOAA (Climatology)
BASE_URL_NOAA = "https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/5km/v3.1_op/climatology/nc/"

# Daftar File
target_files = [
    "ct5km_climatology_v3.1.nc"
]

# Koordinat & Shapefile
lat_min, lat_max = -9, -8
lon_min, lon_max = 115.2, 116.2

# Pastikan file .shp ini BENAR-BENAR ADA di folder Google Drive kamu
regions = {
    "gm": os.path.join(shp_base_dir, "gili_matra_buffer_5km.shp"),
    "gn": os.path.join(shp_base_dir, "gita_nada_buffer_5km.shp"),
    "np": os.path.join(shp_base_dir, "nusa_penida_buffer_5km.shp")
}

# ==========================================
# 2. FUNGSI DOWNLOADER
# ==========================================
def download_noaa_data():
    print("--- CEK KETERSEDIAAN DATA CLIMATOLOGY ---")

    for filename in target_files:
        file_path = os.path.join(input_global_dir, filename)

        if os.path.exists(file_path):
            print(f"✅ {filename} sudah ada di Drive. Skip download.")
        else:
            url = BASE_URL_NOAA + filename
            print(f"⬇️ Sedang mendownload: {filename} ...")
            print(f"   Sumber: {url}")
            print(f"   Target: {file_path}")

            try:
                response = requests.get(url, stream=True)
                response.raise_for_status()

                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)

                print("   -> Download Selesai!")
            except Exception as e:
                print(f"   ❌ GAGAL Download: {e}")

# ==========================================
# 3. FUNGSI PROCESSING
# ==========================================
def process_climatology():
    print("\n--- MULAI PEMROSESAN DATA ---")

    # Cek apakah file ada
    raw_files = glob.glob(os.path.join(input_global_dir, "*.nc"))

    if not raw_files:
        print(f"ERROR: Tidak ada file .nc di {input_global_dir}")
        print("Pastikan download berhasil atau upload manual ke Google Drive.")
        return

    for i, file_path in enumerate(raw_files, 1):
        filename = os.path.basename(file_path)
        filename_no_ext = os.path.splitext(filename)[0]

        print(f"\n[{i}/{len(raw_files)}] Memproses: {filename}")

        try:
            ds = xr.open_dataset(file_path)

            if 'lat' in ds.coords:
                ds = ds.rename({'lat': 'latitude', 'lon': 'longitude'})

            ds = ds.sortby(['latitude', 'longitude'])

            # Handle slicing
            slice_lat = slice(lat_min, lat_max) if ds.latitude[0] < ds.latitude[-1] else slice(lat_max, lat_min)
            ds_lombok = ds.sel(latitude=slice_lat, longitude=slice(lon_min, lon_max))

            if ds_lombok.dims['latitude'] == 0 or ds_lombok.dims['longitude'] == 0:
                print(" -> GAGAL: Area Lombok kosong (Koordinat slicing mungkin salah).")
                continue

            ds_lombok = ds_lombok.rio.set_spatial_dims("longitude", "latitude")
            ds_lombok = ds_lombok.rio.write_crs("EPSG:4326", inplace=True)

            for code, shp_path in regions.items():
                if not os.path.exists(shp_path):
                    print(f"    -> [SKIP] SHP {code} tidak ditemukan di: {shp_path}")
                    continue

                # Baca SHP
                gdf = gpd.read_file(shp_path)
                if gdf.crs != ds_lombok.rio.crs:
                    gdf = gdf.to_crs(ds_lombok.rio.crs)

                try:
                    clipped = ds_lombok.rio.clip(gdf.geometry, gdf.crs, drop=True)
                    df = clipped.to_dataframe().reset_index()

                    ignore_cols = ['latitude', 'longitude', 'spatial_ref', 'crs', 'band', 'time']
                    data_vars = [c for c in df.columns if c not in ignore_cols and c != 'month']

                    if not data_vars: continue
                    target_var = data_vars[0]

                    # Climatology biasanya memiliki variabel 'month' (1-12)
                    if 'month' in df.columns:
                        unique_months = df['month'].unique()
                        for m in unique_months:
                            df_month = df[df['month'] == m]
                            xyz = df_month[['longitude', 'latitude', target_var]].dropna()
                            if xyz.empty: continue

                            # Output XYZ per bulan
                            out_name = f"{code}_{filename_no_ext}_bulan_{int(m)}.xyz"
                            out_path = os.path.join(output_xyz_dir, out_name)
                            xyz.to_csv(out_path, sep=' ', header=False, index=False)
                        print(f"    -> {code}: OK (12 Bulan)")
                    else:
                        # Jika data statis (bukan bulanan)
                        xyz = df[['longitude', 'latitude', target_var]].dropna()
                        if xyz.empty: continue

                        out_name = f"{code}_{filename_no_ext}.xyz"
                        out_path = os.path.join(output_xyz_dir, out_name)
                        xyz.to_csv(out_path, sep=' ', header=False, index=False)
                        print(f"    -> {code}: OK (Statis)")

                except Exception as e_site:
                    print(f"    -> Error Clipping {code}: {e_site}")

            ds.close()
            ds_lombok.close()

        except Exception as e_file:
            print(f" -> ERROR Membuka File {filename}: {e_file}")

    print("\n=== SELESAI SEMUA. CEK GOOGLE DRIVE KAMU ===")

if __name__ == "__main__":
    # Jalankan Downloader
    download_noaa_data()
    # Jalankan Processing
    process_climatology()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- CEK KETERSEDIAAN DATA CLIMATOLOGY ---
⬇️ Sedang mendownload: ct5km_climatology_v3.1.nc ...
   Sumber: https://www.star.nesdis.noaa.gov/pub/sod/mecb/crw/data/5km/v3.1_op/climatology/nc/ct5km_climatology_v3.1.nc
   Target: /content/drive/MyDrive/MAGANG/CORAL/01_climatology/00_Raw_Climatology/ct5km_climatology_v3.1.nc
   -> Download Selesai!

--- MULAI PEMROSESAN DATA ---

[1/1] Memproses: ct5km_climatology_v3.1.nc
    -> gm: OK (Statis)
    -> gn: OK (Statis)
    -> np: OK (Statis)

=== SELESAI SEMUA. CEK GOOGLE DRIVE KAMU ===


### Tahap 2: Perhitungan Statistik Climatology (MMM)
**Tujuan:** Menghitung rata-rata bulanan dan nilai MMM (Maximum Monthly Mean) yang menjadi ambang batas bleaching.

**Output:**
Script ini akan menghasilkan file teks penting bernama: **`mmm_mean_site_FINAL.txt`**.
Pastikan file ini berhasil muncul di folder `01_climatology` sebelum lanjut ke tahap akhir.

In [6]:
import os
import xarray as xr
import rioxarray
import geopandas as gpd
import pandas as pd
import numpy as np
import warnings
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Abaikan warning
warnings.filterwarnings("ignore")

# ==========================================
# 1. KONFIGURASI PATH (GOOGLE DRIVE)
# ==========================================

# Sesuaikan path ini dengan lokasi di Google Drive kamu
base_dir = "/content/drive/MyDrive/MAGANG/CORAL/01_climatology"

# File input (Pastikan file ini sudah didownload oleh script sebelumnya)
input_nc = os.path.join(base_dir, "00_Raw_Climatology", "ct5km_climatology_v3.1.nc")
shp_base_dir = os.path.join(base_dir, "SHP_SITE")

# File Output Laporan
output_report = os.path.join(base_dir, "mmm_mean_site_FINAL.txt")

# Koordinat Kotak Lombok (Slicing agar RAM Aman)
lat_min, lat_max = -9.2, -8.2
lon_min, lon_max = 115.3, 116.3

# Definisi Lokasi Shapefile
regions = {
    "gm": os.path.join(shp_base_dir, "gili_matra_buffer_5km.shp"),
    "gn": os.path.join(shp_base_dir, "gita_nada_buffer_5km.shp"),
    "np": os.path.join(shp_base_dir, "nusa_penida_buffer_5km.shp")
}

# DAFTAR NAMA VARIABEL BULAN (Sesuai standar NOAA CRW v3.1)
month_vars = [
    'sst_clim_january', 'sst_clim_february', 'sst_clim_march',
    'sst_clim_april', 'sst_clim_may', 'sst_clim_june',
    'sst_clim_july', 'sst_clim_august', 'sst_clim_september',
    'sst_clim_october', 'sst_clim_november', 'sst_clim_december'
]

def calculate_site_climatology():
    # Cek keberadaan file NetCDF
    if not os.path.exists(input_nc):
        print(f"❌ File Climatology tidak ditemukan di: {input_nc}")
        print("   Pastikan kamu sudah menjalankan script download sebelumnya.")
        return

    print(f"📂 Membuka dataset: {os.path.basename(input_nc)}")

    try:
        ds = xr.open_dataset(input_nc)

        # --- TAHAP 1: NORMALISASI & SLICING ---
        # Rename koordinat jika perlu
        if 'lat' in ds.coords:
            ds = ds.rename({'lat': 'latitude', 'lon': 'longitude'})
        ds = ds.sortby(['latitude', 'longitude'])

        print("✂️  Memotong area Lombok (Saving RAM)...")

        # Slicing Spasial (Handle urutan latitude ascending/descending)
        if ds.latitude[0] > ds.latitude[-1]:
            ds_lombok = ds.sel(latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max))
        else:
            ds_lombok = ds.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))

        # Set CRS (Coordinate Reference System)
        ds_lombok = ds_lombok.rio.set_spatial_dims("longitude", "latitude").rio.write_crs("EPSG:4326")

        results = []
        print("\n--- MULAI PERHITUNGAN SITE ---")

        for code, shp_path in regions.items():
            # Cek apakah shapefile ada
            if not os.path.exists(shp_path):
                print(f"⚠️  Skip {code}: Shapefile tidak ditemukan ({shp_path})")
                continue

            print(f"🔄 Processing Site: {code.upper()}")

            try:
                gdf = gpd.read_file(shp_path)

                # Samakan CRS Shapefile dengan NetCDF
                if gdf.crs != ds_lombok.rio.crs:
                    gdf = gdf.to_crs(ds_lombok.rio.crs)

                # 1. Clip NetCDF sesuai bentuk Shapefile
                clipped = ds_lombok.rio.clip(gdf.geometry, gdf.crs, drop=True)

                site_means = []

                # 2. LOOP 12 BULAN
                for var_name in month_vars:
                    if var_name in clipped.data_vars:
                        # Hitung rata-rata spasial (spatial mean) area tersebut
                        val = clipped[var_name].mean(dim=['latitude', 'longitude'], skipna=True).item()
                        site_means.append(val)
                    else:
                        print(f"   ⚠️ Warning: Variabel {var_name} tidak ada.")
                        site_means.append(np.nan)

                # 3. Hitung MMM (Maximum Monthly Mean)
                clean_means = [x for x in site_means if not np.isnan(x)]

                if len(clean_means) == 12:
                    mmm_value = max(clean_means) # Nilai tertinggi dari 12 bulan

                    results.append({
                        'name': code.upper(),
                        'mmm': mmm_value,
                        'means': clean_means
                    })
                    print(f"   ✅ OK. MMM: {mmm_value:.4f}")
                else:
                    print("   ❌ Gagal: Data bulan tidak lengkap.")

            except Exception as e:
                print(f"   ❌ Error pada {code}: {e}")

        # --- TAHAP 3: SIMPAN OUTPUT ---
        if results:
            with open(output_report, 'w') as f:
                for res in results:
                    f.write(f"SITE: {res['name']}\n")
                    f.write("Averaged Maximum Monthly Mean:\n")
                    f.write(f"{res['mmm']:.4f}\n\n")

                    f.write("Averaged Monthly Mean (Jan-Dec):\n")
                    # Join list menjadi string
                    means_str = " ".join([f"{val:.4f}" for val in res['means']])
                    f.write(f"{means_str}\n")

                    f.write("\n" + "="*40 + "\n\n")

            print(f"\n🎉 SELESAI! Laporan tersimpan di:\n   {output_report}")
            print("   Kamu bisa menggunakan file ini untuk script Final Report nanti.")
        else:
            print("\n⚠️ Tidak ada hasil yang disimpan (Cek apakah Shapefile/NetCDF valid).")

        ds.close()
        ds_lombok.close()

    except Exception as e_open:
        print(f"CRITICAL ERROR: {e_open}")

if __name__ == "__main__":
    calculate_site_climatology()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Membuka dataset: ct5km_climatology_v3.1.nc
✂️  Memotong area Lombok (Saving RAM)...

--- MULAI PERHITUNGAN SITE ---
🔄 Processing Site: GM
   ✅ OK. MMM: 29.0514
🔄 Processing Site: GN
   ✅ OK. MMM: 28.6450
🔄 Processing Site: NP
   ✅ OK. MMM: 28.4726

🎉 SELESAI! Laporan tersimpan di:
   /content/drive/MyDrive/MAGANG/CORAL/01_climatology/mmm_mean_site_FINAL.txt
   Kamu bisa menggunakan file ini untuk script Final Report nanti.


# 90th percentile Coral Bleaching

In [7]:
# Install library geospatial yang diperlukan
!pip install xarray rioxarray geopandas netCDF4

In [15]:
import pandas as pd
import numpy as np
import os
import datetime
import re
from collections import deque
import xarray as xr
import rioxarray
import geopandas as gpd
import warnings
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Abaikan warning
warnings.filterwarnings("ignore")

# ==============================================================================
# 1. KONFIGURASI PATH & FILE (GOOGLE DRIVE)
# ==============================================================================

# --- TENTUKAN ROOT FOLDER PROYEK ---
# Sesuaikan 'MAGANG/CORAL/output' dengan lokasi folder kamu di Drive
PROJECT_ROOT = "/content/drive/MyDrive/MAGANG/CORAL"

# --- KONFIGURASI CLIMATOLOGY (INPUT) ---
# Folder 01_climatology ada di dalam PROJECT_ROOT
base_dir_clim = os.path.join(PROJECT_ROOT, "01_climatology")

input_nc = os.path.join(base_dir_clim, "00_Raw_Climatology", "ct5km_climatology_v3.1.nc")
shp_base_dir = os.path.join(base_dir_clim, "SHP_SITE")

# Mapping Kode Wilayah ke File Shapefile
regions_shp = {
    "GM": os.path.join(shp_base_dir, "gili_matra_buffer_5km.shp"),
    "GN": os.path.join(shp_base_dir, "gita_nada_buffer_5km.shp"),
    "NP": os.path.join(shp_base_dir, "nusa_penida_buffer_5km.shp")
}

# --- KONFIGURASI DATA HARIAN (INPUT) ---
# Pastikan folder "03_Masking_Site" ada di dalam PROJECT_ROOT atau sesuaikan path-nya
INPUT_FOLDER_XYZ = os.path.join(PROJECT_ROOT, "03_Masking_Site")

# --- KONFIGURASI OUTPUT ---
OUTPUT_FOLDER = os.path.join(PROJECT_ROOT, "NOAA_Final_Reports_Integrated")

# Variabel Bulan dalam NetCDF
month_vars = [
    'sst_clim_january', 'sst_clim_february', 'sst_clim_march',
    'sst_clim_april', 'sst_clim_may', 'sst_clim_june',
    'sst_clim_july', 'sst_clim_august', 'sst_clim_september',
    'sst_clim_october', 'sst_clim_november', 'sst_clim_december'
]

# Koordinat Slicing (Lombok Area)
lat_min, lat_max = -9.2, -8.2
lon_min, lon_max = 115.3, 116.3

# ==============================================================================
# 2. MODUL CLIMATOLOGY (XARRAY & GEOPANDAS)
# ==============================================================================
def calculate_climatology_data():
    """
    Menghitung MMM dan Monthly Mean dari file NetCDF menggunakan Shapefile.
    Mengembalikan dictionary berisi data klimatologi per site.
    """
    print("\n" + "="*50)
    print("   MEMULAI PERHITUNGAN CLIMATOLOGY (NetCDF)")
    print("="*50)

    if not os.path.exists(input_nc):
        print(f"❌ File NC tidak ditemukan: {input_nc}")
        print("   Pastikan path di Google Drive sudah benar.")
        return {}

    # Buka Dataset
    try:
        ds = xr.open_dataset(input_nc)

        # Normalisasi nama dimensi
        if 'lat' in ds.coords: ds = ds.rename({'lat': 'latitude', 'lon': 'longitude'})
        ds = ds.sortby(['latitude', 'longitude'])

        # Slicing Area Lombok (Hemat RAM)
        print("✂️  Memotong area Lombok...")
        if ds.latitude[0] > ds.latitude[-1]:
            ds_lombok = ds.sel(latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max))
        else:
            ds_lombok = ds.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))

        ds_lombok = ds_lombok.rio.set_spatial_dims("longitude", "latitude").rio.write_crs("EPSG:4326")

        clim_results = {}

        for code, shp_path in regions_shp.items():
            if not os.path.exists(shp_path):
                print(f"⚠️  Shapefile tidak ditemukan untuk {code}: {shp_path}")
                continue

            print(f"🔄 Processing Climatology: {code}")
            gdf = gpd.read_file(shp_path)

            # Samakan CRS
            if gdf.crs != ds_lombok.rio.crs:
                gdf = gdf.to_crs(ds_lombok.rio.crs)

            try:
                # Clip NetCDF dengan Shapefile
                clipped = ds_lombok.rio.clip(gdf.geometry, gdf.crs, drop=True)

                site_means = []
                # Loop 12 Bulan
                for var_name in month_vars:
                    if var_name in clipped.data_vars:
                        val = clipped[var_name].mean(dim=['latitude', 'longitude'], skipna=True).item()
                        site_means.append(val)
                    else:
                        site_means.append(np.nan)

                clean_means = [x for x in site_means if not np.isnan(x)]

                if len(clean_means) == 12:
                    mmm_value = max(clean_means)
                    clim_results[code] = {
                        "mmm": mmm_value,
                        "monthly_means": clean_means
                    }
                    print(f"   ✅ {code} MMM: {mmm_value:.4f}")
                else:
                    print(f"   ❌ {code}: Data bulan tidak lengkap.")

            except Exception as e:
                print(f"   ❌ Error processing {code}: {e}")

        ds.close()
        return clim_results

    except Exception as e:
        print(f"CRITICAL ERROR CLIMATOLOGY: {e}")
        return {}

# ==============================================================================
# 3. MODUL HARIAN (PANDAS & NUMPY) -- FIXED BAA LOGIC --
# ==============================================================================
class RegionAnalyzer:
    def __init__(self, name, code, climatology_data):
        self.name = name
        self.code = code
        self.stress_window = deque(maxlen=84) # 12 minggu untuk DHW
        self.baa_window = deque(maxlen=7)     # 7 hari untuk BAA Composite
        self.center_lat = 0.0
        self.center_lon = 0.0
        self.coord_set = False

        # Data Climatology dari Tahap 1
        self.mmm = climatology_data.get('mmm', 0.0)
        self.monthly_means = climatology_data.get('monthly_means', [0.0]*12)

    def process_day(self, date_obj, file_hs, file_sst=None, file_ssta=None):
        try:
            # 1. Baca HS
            df_hs = pd.read_csv(file_hs, sep='\s+', header=None, names=['lon', 'lat', 'val'])
            df_hs = df_hs.dropna()
            if df_hs.empty: return None

            # Ambil koordinat pusat (Rata-rata seluruh piksel dalam mask)
            if not self.coord_set:
                self.center_lon = df_hs['lon'].mean()
                self.center_lat = df_hs['lat'].mean()
                self.coord_set = True

            # 2. Hitung 90th Percentile HS
            hs_values = df_hs['val'].values
            hs_90 = np.percentile(hs_values, 90)

            # Cari indeks piksel untuk mengambil SST di lokasi yang sama
            idx_p90 = (np.abs(hs_values - hs_90)).argmin()

            # 3. Ambil SST & SSTA (Jika ada)
            sst_val = -999.0
            sst_min = -999.0
            sst_max = -999.0
            ssta_val = -999.0

            if file_sst and os.path.exists(file_sst):
                try:
                    df_sst = pd.read_csv(file_sst, sep='\s+', header=None, names=['lon', 'lat', 'val'])
                    sst_val = df_sst['val'].iloc[idx_p90]
                    sst_min = df_sst['val'].min()
                    sst_max = df_sst['val'].max()
                except: pass

            if file_ssta and os.path.exists(file_ssta):
                try:
                    df_ssta = pd.read_csv(file_ssta, sep='\s+', header=None, names=['lon', 'lat', 'val'])
                    ssta_val = df_ssta['val'].iloc[idx_p90]
                except: pass

            # 4. Hitung DHW (Akumulasi)
            daily_stress = 0.0
            if hs_90 >= 1.0:
                daily_stress = hs_90 / 7.0

            self.stress_window.append(daily_stress)
            current_dhw = sum(self.stress_window)

            # 5. Hitung BAA (Logika Baru: Instantaneous -> 7-Day Max)
            # Tentukan level alert instan hari ini
            # 0: No Stress, 1: Watch, 2: Warning, 3: Alert Lvl 1, 4: Alert Lvl 2
            daily_alert_level = 0

            if hs_90 <= 0.0:
                daily_alert_level = 0 # No Stress
            elif 0.0 < hs_90 < 1.0:
                daily_alert_level = 1 # Watch
            else: # hs_90 >= 1.0
                if current_dhw < 4.0:
                    daily_alert_level = 2 # Warning (Possible Bleaching)
                elif 4.0 <= current_dhw < 8.0:
                    daily_alert_level = 3 # Alert Level 1
                elif current_dhw >= 8.0:
                    daily_alert_level = 4 # Alert Level 2

            # Masukkan ke window 7 hari
            self.baa_window.append(daily_alert_level)

            # Ambil nilai maksimum dalam 7 hari terakhir (Composite)
            final_baa = max(self.baa_window) if self.baa_window else 0

            return {
                "date": date_obj,
                "sst_min": sst_min,
                "sst_max": sst_max,
                "sst_90": sst_val,
                "ssta_90": ssta_val,
                "hs_90": max(0, hs_90),
                "dhw": current_dhw,
                "baa": final_baa # Menggunakan hasil composite
            }
        except Exception as e:
            print(f"Error reading daily file {file_hs}: {e}")
            return None

# ==============================================================================
# 4. MAIN EXECUTION
# ==============================================================================
def main():
    if not os.path.exists(OUTPUT_FOLDER): os.makedirs(OUTPUT_FOLDER)

    # --- TAHAP 1: HITUNG CLIMATOLOGY ---
    clim_data_store = calculate_climatology_data()

    if not clim_data_store:
        print("⚠️  Peringatan: Data Climatology Kosong/Gagal. Melanjutkan dengan nilai default 0.")

    # --- TAHAP 2: INVENTARISASI FILE HARIAN ---
    print("\n" + "="*50)
    print(f"   MEMULAI ANALISIS HARIAN (Folder: {INPUT_FOLDER_XYZ})")
    print("="*50)

    if not os.path.exists(INPUT_FOLDER_XYZ):
        print(f"❌ Folder Input XYZ tidak ditemukan: {INPUT_FOLDER_XYZ}")
        return

    files_map = {}

    # Scanning folder
    for f in os.listdir(INPUT_FOLDER_XYZ):
        if not f.endswith(".xyz"): continue

        name_lower = f.lower()
        if "np_" in name_lower: region = "NP"
        elif "gm_" in name_lower: region = "GM"
        elif "gn_" in name_lower: region = "GN"
        else: continue

        date_match = re.search(r"(\d{8})", f)
        if not date_match: continue
        date_str = date_match.group(1)

        ftype = "UNKNOWN"
        if "hs" in name_lower and "hotspot" not in name_lower: ftype = "HS"
        elif "hotspot" in name_lower: ftype = "HS"
        elif "ssta" in name_lower: ftype = "SSTA"
        elif "sst" in name_lower: ftype = "SST"

        if region not in files_map: files_map[region] = {}
        if date_str not in files_map[region]: files_map[region][date_str] = {}

        files_map[region][date_str][ftype] = os.path.join(INPUT_FOLDER_XYZ, f)

    # --- TAHAP 3: PROSES DAN TULIS LAPORAN ---
    full_names = {"NP": "Nusa Penida", "GM": "Gili Matra", "GN": "Gita Nada"}

    for code, dates_dict in files_map.items():
        region_fullname = full_names.get(code, code)
        print(f"\n📈 Memproses Wilayah: {region_fullname} ({code})")

        # Ambil data climatology spesifik untuk region ini
        site_clim = clim_data_store.get(code, {'mmm': 0.0, 'monthly_means': [0.0]*12})

        analyzer = RegionAnalyzer(region_fullname, code, site_clim)
        sorted_dates = sorted(dates_dict.keys())
        results_buffer = []

        # Loop Harian
        for d_str in sorted_dates:
            files = dates_dict[d_str]
            if "HS" not in files: continue

            dt_obj = datetime.datetime.strptime(d_str, "%Y%m%d")
            data = analyzer.process_day(
                dt_obj,
                files["HS"],
                files.get("SST"),
                files.get("SSTA")
            )
            if data: results_buffer.append(data)

        if not results_buffer: continue

        # Menulis File Output
        output_filename = os.path.join(OUTPUT_FOLDER, f"{region_fullname.replace(' ','_')}_NOAA_Combined.txt")

        with open(output_filename, "w") as f:
            # HEADER
            f.write("Name:\n")
            f.write(f"{region_fullname}\n\n")
            f.write("Polygon Middle Longitude:\n")
            f.write(f"{analyzer.center_lon:.4f} \n\n")
            f.write("Polygon Middle Latitude:\n")
            f.write(f"{analyzer.center_lat:.4f} \n\n")

            # --- DATA CLIMATOLOGY ---
            f.write("Averaged Maximum Monthly Mean:\n")
            f.write(f"{analyzer.mmm:.4f}\n\n")

            f.write("Averaged Monthly Mean (Jan-Dec):\n")
            means_str = " ".join([f"{val:.4f}" for val in analyzer.monthly_means])
            f.write(f"{means_str}\n\n")

            # --- TANGGAL VALID ---
            first_dt = results_buffer[0]['date']
            dhw_valid_dt = first_dt + datetime.timedelta(weeks=12)
            baa_valid_dt = dhw_valid_dt + datetime.timedelta(days=7)

            f.write("First Valid DHW Date:\n")
            f.write(f"{dhw_valid_dt.strftime('%Y %m %d')}\n\n")
            f.write("First Valid BAA Date:\n")
            f.write(f"{baa_valid_dt.strftime('%Y %m %d')}\n\n")

            # TABEL DATA
            header = "YYYY MM DD SST_MIN SST_MAX SST@90th_HS SSTA@90th_HS 90th_HS>0 DHW_from_90th_HS>1 BAA_7day_max"
            f.write(header + "\n")

            for row in results_buffer:
                d = row['date']
                line = (
                    f"{d.year:4d} {d.month:02d} {d.day:02d} "
                    f"{row['sst_min']:7.4f} {row['sst_max']:7.4f} "
                    f"{row['sst_90']:11.4f} {row['ssta_90']:12.4f} "
                    f"{row['hs_90']:9.4f} "
                    f"{row['dhw']:18.4f} "
                    f"{row['baa']:12d}"
                )
                f.write(line + "\n")

        print(f"✅ Laporan tersimpan: {output_filename}")

    print("\n" + "="*50)
    print("PROSES SELESAI SEMUA")
    print("="*50)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

   MEMULAI PERHITUNGAN CLIMATOLOGY (NetCDF)
✂️  Memotong area Lombok...
🔄 Processing Climatology: GM
   ✅ GM MMM: 29.0514
🔄 Processing Climatology: GN
   ✅ GN MMM: 28.6450
🔄 Processing Climatology: NP
   ✅ NP MMM: 28.4726

   MEMULAI ANALISIS HARIAN (Folder: /content/drive/MyDrive/MAGANG/CORAL/03_Masking_Site)

📈 Memproses Wilayah: Gili Matra (GM)
✅ Laporan tersimpan: /content/drive/MyDrive/MAGANG/CORAL/NOAA_Final_Reports_Integrated/Gili_Matra_NOAA_Combined.txt

📈 Memproses Wilayah: Gita Nada (GN)
✅ Laporan tersimpan: /content/drive/MyDrive/MAGANG/CORAL/NOAA_Final_Reports_Integrated/Gita_Nada_NOAA_Combined.txt

📈 Memproses Wilayah: Nusa Penida (NP)
✅ Laporan tersimpan: /content/drive/MyDrive/MAGANG/CORAL/NOAA_Final_Reports_Integrated/Nusa_Penida_NOAA_Combined.txt

PROSES SELESAI SEMUA


In [25]:
import shutil
import os
from google.colab import files

# 1. Tentukan folder mana yang mau di-zip (SUMBER)
folder_to_zip = "/content/drive/MyDrive/MAGANG/CORAL/NOAA_Final_Reports"

# 2. Tentukan nama file zip-nya (TUJUAN)
# Jangan gunakan nama yang sama persis dengan folder aslinya agar tidak bingung
# File akan disimpan di folder CORAL dengan nama "02_Clip_lombok_DOWNLOAD.zip"
zip_destination = "/content/drive/MyDrive/MAGANG/CORAL/NOAA_Final_Reports"

# 3. Eksekusi
print(f"📦 Sedang men-zip folder: {folder_to_zip} ...")

# make_archive(nama_file_tujuan, format, folder_sumber)
shutil.make_archive(zip_destination, 'zip', folder_to_zip)

print(f"✅ Selesai! File tersimpan: {zip_destination}.zip")

# 4. Download
print("⬇️ Mendownload ke Laptop...")
files.download(f"{zip_destination}.zip")

📦 Sedang men-zip folder: /content/drive/MyDrive/MAGANG/CORAL/NOAA_Final_Reports ...
✅ Selesai! File tersimpan: /content/drive/MyDrive/MAGANG/CORAL/NOAA_Final_Reports.zip
⬇️ Mendownload ke Laptop...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>